In [1]:
from datasets import load_dataset

# Load the Fineweb-2 Dataset (Myanmar subset including removed)

In [2]:
# get Croatian data
ds_mya = load_dataset("HuggingFaceFW/fineweb-2", name="mya_Mymr")
ds_mya_removed = load_dataset("HuggingFaceFW/fineweb-2", name="mya_Mymr_removed")

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

In [3]:
ds_mya['train'][0]

{'text': 'ျပသဆဲႏွင့္ ႐ံုတင္မည့္ ႏုိင္ငံျခား႐ုပ္ရွင္မိတ္ဆက္\nေထာက္လွမ္းေရးေအဂ်င္စီ S.H.I.E.L.D၏ ညႊန္ၾကားေရးမႉး နစ္ျဖဴရီသည္ ေဝးလံေခါင္သီေသာ အရပ္ရွိ သုေတသနစခန္းသို႔ ေရာက္ သြားရာ တက္ဆရက္အမည္ရွိ စြမ္းအင္အရင္းအျမစ္တစ္ခုေၾကာင့္ အာကာသစၾကဝဠာထဲမွ ဝင္ေပါက္ ႀကီးတစ္ခုပြင့္ၿပီး ျပည္ႏွင္ ဒဏ္ေပးခံ ထားရေသာ ေနာ့(စ္)နတ္ဘုရား လိုကီက ထိုဝင္ေပါက္မွ ဝင္လာကာ တက္ဆရက္ကို ရယူသြားသည္။ လိုကီသည္ ထြက္ေျပးသည့္အတြက္ ႐ူပေဗဒပညာရွင္ ေဒါက္တာဆဲလ္ဗစ္ တို႔အပါအဝင္ S.H.I.E.L.D အမႈ ထမ္းအေတာ္မ်ားမ်ား၏စိတ္ကို ထိန္းခ်ဳပ္ခိုင္းေစလ်က္ရွိသည္။ ၿမိဳ႕ေတာ္ထြန္းရည္\nထို႔ေၾကာင့္ ျဖဴရီက လိုကီ့လက္ ထဲမွတက္ဆရက္ကို ျပန္ယူဖို႔ စတိဗ္ ေရာဂ်ား(စ္) ကို တာဝန္ေပးသည္။ လိုကီက ခ်ီေတာရီးေခၚ ၿဂဳိဟ္သား မ်ားႏွင့္ မဟာမိတ္ဖြဲ႕ၿပီး ၄င္းတို႔အား တက္ဆရက္ကိုေပးအပ္ကာ ကမၻာကို အုပ္စိုးရန္ အကူအညီေတာင္းသည္။ လိုကီႏွင့္ လူစြမ္းေကာင္းမ်ား တိုက္ပြဲ ျဖစ္ၿပီးေနာက္ လိုကီက အညံ့ခံသျဖင့္ S.H.I.E.L.D ေအဂ်င္စီပိုင္ေလ ယာဥ္ျဖင့္ သူ႔ကိုဖမ္းေခၚသြားၾက သည္။ လိုကီ၏ ေမြးစားညီငယ္ မိုးႀကဳိးနတ္ ဘုရားေသာ္(ရ္)က လိုကီကို အခ်ဳပ္မွ လႊတ္ေပးရန္ႀကဳိးစားသည္။ သံမဏိ လူသား၊ ကပၸတိန္အေမရိကတို႔ႏွင့္ ရင္ဆိုင

In [4]:
from myanmartools import ZawgyiDetector
from icu import Transliterator

# Example Convertion from Zawgyi to Unicode

In [7]:
detector = ZawgyiDetector()
converter = Transliterator.createInstance('Zawgyi-my')

original_text = ds_mya['train'][0]['text'].split("\n")[0]
score = detector.get_zawgyi_probability(original_text)

print("Original Text:", original_text)

if score > 0.5:
    converted_text = converter.transliterate(original_text)
    print()
    print("Score:", score)
    print()
    print("Converted Text:", converted_text)

Original Text: ျပသဆဲႏွင့္ ႐ံုတင္မည့္ ႏုိင္ငံျခား႐ုပ္ရွင္မိတ္ဆက္

Score: 1.0

Converted Text: ပြသဆဲနှင့် ရုံတင်မည့် နိုင်ငံခြားရုပ်ရှင်မိတ်ဆက်


# Convert the Whole Dataset from Zawgyi to Unicode

In [8]:
def zawgyi_to_unicode_converter(examples):
    detector = ZawgyiDetector()
    converter = Transliterator.createInstance('Zawgyi-my')

    articles = examples['text']
    converted_articles = []

    for article in articles:
        
        converted_chunks = []
        for text_chunk in article.split("\n"):
            score = detector.get_zawgyi_probability(text_chunk)
            if score > 0.5:
                text_chunk = converter.transliterate(text_chunk)

            converted_chunks.append(text_chunk)
        
        converted_articles.append("\n".join(converted_chunks))

    examples['text'] = converted_articles
    return examples

In [9]:
ds_mya_unicode = ds_mya['train'].map(
    zawgyi_to_unicode_converter,
    batched=True,
    batch_size=100,
    num_proc=10
)

Map (num_proc=10):   0%|          | 0/1619895 [00:00<?, ? examples/s]

In [11]:
ds_mya_unicode_test = ds_mya['test'].map(
    zawgyi_to_unicode_converter,
    batched=True,
    batch_size=100,
    num_proc=10
)

Map (num_proc=10):   0%|          | 0/16096 [00:00<?, ? examples/s]

In [10]:
ds_mya_removed_unicode = ds_mya_removed['train'].map(
    zawgyi_to_unicode_converter,
    batched=True,
    batch_size=100,
    num_proc=10
)

Map (num_proc=10):   0%|          | 0/1033074 [00:00<?, ? examples/s]

In [14]:
ds_mya_removed_unicode

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'filter_reason', 'top_langs'],
    num_rows: 1033074
})

# Create the Dataset

In [15]:
from datasets import Dataset, DatasetDict

In [16]:
non_removed_dataset = DatasetDict({
    "train": ds_mya_unicode,
    "test": ds_mya_unicode_test
})

# Create the second configuration (removed)
removed_dataset = DatasetDict({
    "train": ds_mya_removed_unicode,
})

In [17]:
non_removed_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_rows: 1619895
    })
    test: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_rows: 16096
    })
})

In [18]:
removed_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'filter_reason', 'top_langs'],
        num_rows: 1033074
    })
})

## Upload to HuggingFace

In [23]:
non_removed_dataset.push_to_hub("chuuhtetnaing/myanmar-fineweb-2-dataset", config_name="filtered")
removed_dataset.push_to_hub("chuuhtetnaing/myanmar-fineweb-2-dataset", config_name="removed")

Uploading the dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/87 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/87 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/87 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/801 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chuuhtetnaing/myanmar-fineweb-2-dataset/commit/8eaa0747bb878cb7f7d9bc28ea170f75e414c126', commit_message='Upload dataset', commit_description='', oid='8eaa0747bb878cb7f7d9bc28ea170f75e414c126', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chuuhtetnaing/myanmar-fineweb-2-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chuuhtetnaing/myanmar-fineweb-2-dataset'), pr_revision=None, pr_num=None)

# Load the Uploaded Dataset

In [27]:
from datasets import concatenate_datasets

In [30]:
removed_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'filter_reason', 'top_langs'],
        num_rows: 1033074
    })
})

In [31]:
filtered_ds = load_dataset("chuuhtetnaing/myanmar-fineweb-2-dataset", name='filtered')
removed_ds = load_dataset("chuuhtetnaing/myanmar-fineweb-2-dataset", name='removed')

ds = DatasetDict({
    "train": concatenate_datasets([filtered_ds["train"], removed_ds["train"]]),
    "test": filtered_ds["test"]
})

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/29 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [32]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs', 'filter_reason'],
        num_rows: 2652969
    })
    test: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_rows: 16096
    })
})